<a href="https://colab.research.google.com/github/FCam4/Proj_6_Deep_Learning/blob/main/ml_lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Algorithms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# Regularization
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# Evaluation Metrics
from math import sqrt
from sklearn.metrics import SCORERS
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Feature Scalling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_1 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore1.csv')
df_2 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore2.csv')
df_3 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore3.csv')
df_4 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore4.csv')
df_5 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore5.csv')
df_6 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore6.csv')
df_7 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore7.csv')
df_8 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore8.csv')
df_9 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore9.csv')
df_10 = pd.read_csv('https://raw.githubusercontent.com/FCam4/Proj_6_Deep_Learning/main/Google_Playstore10.csv')
df_12 = pd.concat([df_1, df_2], axis=0)
df_123 = pd.concat([df_12, df_3], axis=0)
df_1234 = pd.concat([df_123, df_4], axis=0)
df_12345 = pd.concat([df_1234, df_5], axis=0)
df_123456 = pd.concat([df_12345, df_6], axis=0)
df_1234567 = pd.concat([df_123456, df_7], axis=0)
df_12345678 = pd.concat([df_1234567, df_8], axis=0)
df_123456789 = pd.concat([df_12345678, df_9], axis=0)
df_pstore = pd.concat([df_123456789, df_10], axis=0)

In [ ]:
df_pstore_ml = df_pstore.dropna(subset=['App Name','Rating', 'Rating Count', 'Installs', 'Minimum Installs', 'Maximum Installs', 'Size', 'Developer Id'])
df_pstore_ml = pd.get_dummies(df_pstore_ml, columns=['Category'])
condition = df_pstore_ml[(df_pstore_ml['Size'] == 'Varies with device')].index
df_pstore_ml.drop(condition , inplace=True)

In [ ]:
substring = "M"
def turn_M_to_bite (value):
  substring_m = "M"
  substring_k = "k"
  substring_g = "G"
  if substring_m in value:
    value = value.replace(substring_m, "")
    value = value.replace(",", ".")
    value = float(value)
    value = value*1000000
  elif substring_k in value: 
    value = value.replace(substring_k, "")
    value = value.replace(",", ".")
    # print(value)
    value = float(value)
    value = value*1000
  elif substring_g in value: 
    value = value.replace(substring_g, "")
    value = value.replace(",", ".")
    # print(value)
    value = float(value)
    value = value*1000000000
  else:
    return value
  return value

df_pstore_ml['Size'] = df_pstore_ml['Size'].apply(turn_M_to_bite)
df_pstore_ml['Size'].value_counts()

11000000.0     50441
12000000.0     45608
13000000.0     39022
14000000.0     36568
16000000.0     34556
               ...  
351000000.0        1
282000000.0        1
878000000.0        1
426000000.0        1
512000000.0        1
Name: Size, Length: 1621, dtype: int64

In [ ]:
X = df_pstore_ml[['Rating Count', 'Maximum Installs', 'Ad Supported', 'Size', 'Category_Action', 'Category_Adventure',
       'Category_Arcade', 'Category_Art & Design', 'Category_Auto & Vehicles',
       'Category_Beauty', 'Category_Board', 'Category_Books & Reference',
       'Category_Business', 'Category_Card', 'Category_Casino',
       'Category_Casual', 'Category_Comics', 'Category_Communication',
       'Category_Dating', 'Category_Education', 'Category_Educational',
       'Category_Entertainment', 'Category_Events', 'Category_Finance',
       'Category_Food & Drink', 'Category_Health & Fitness',
       'Category_House & Home', 'Category_Libraries & Demo',
       'Category_Lifestyle', 'Category_Maps & Navigation', 'Category_Medical',
       'Category_Music', 'Category_Music & Audio', 'Category_News & Magazines',
       'Category_Parenting', 'Category_Personalization',
       'Category_Photography', 'Category_Productivity', 'Category_Puzzle',
       'Category_Racing', 'Category_Role Playing', 'Category_Shopping',
       'Category_Simulation', 'Category_Social', 'Category_Sports',
       'Category_Strategy', 'Category_Tools', 'Category_Travel & Local',
       'Category_Trivia', 'Category_Video Players & Editors',
       'Category_Weather', 'Category_Word']]
y = df_pstore_ml['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state=19)

In [ ]:
lasso_hyp = {
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    }
    
pipe_hyp = {
    'scaler__with_mean': [True, False],
    'scaler__with_std': [True, False],
    'lasso__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    } 

pipe_hyp2 = {
    'lasso__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    } 
    
all_options_dict = {
    'models': [Lasso(), Pipeline([('scaler', StandardScaler()), ('Lasso', Lasso())]), Pipeline([('minmax', MinMaxScaler()), ('Lasso', Lasso())])],
    'hyper': [lasso_hyp, pipe_hyp, pipe_hyp2]
}

all_options_dataframe = pd.DataFrame().from_dict(all_options_dict, orient = 'columns')
all_options_dataframe

,models,hyper
0,Lasso(),"{'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, ..."
1,"(StandardScaler(), Lasso())","{'scaler__with_mean': [True, False], 'scaler__..."
2,"(MinMaxScaler(), Lasso())","{'lasso__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6..."


In [ ]:
def grid_search_func(mdl, hyp):
  gs = GridSearchCV(mdl, hyp, scoring = 'accuracy')
  gs.fit(X_train, y_train)
  return gs.best_params_, gs.best_score_

all_options_dataframe[['best params', 'best score']] = all_options_dataframe.apply(lambda row: grid_search_func(row['models'], row['hyper']), result_type = 'expand', axis = 1)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 104, in _check_targets
    raise ValueError("{0} is not 

ValueError: ignored

In [ ]:
all_options_dataframe.sort_values('best score', ascending = False)

In [ ]:
from sklearn.ensemble import VotingRegressor
# from mlxtend.regressor import StackingCVRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# reg1 = DecisionTreeRegressor()
# reg2 = LinearRegression()
# reg3 = KNeighborsRegressor()

# Create voting regressor
reg_gradboost = GradientBoostingRegressor(random_state=0)

# Fit and predict with the models and ensemble
# algorithms_list = [reg1, reg2, reg3, voting_ens]
# algorithms_list = list(voting_ens)
# def voting_regressor_func(algorithm):
  # print(algorithm.__class__.__name__)
reg_gradboost.fit(X_train, y_train)
print("Training score", reg_gradboost.score(X_train, y_train))
print("Test score", reg_gradboost.score(X_test, y_test))

Training score 0.9539745436190443
Test score 0.9541936531647478
